In [1]:
%run __init__.py
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d

In [4]:
calib_extrinsics = np.load("/home/konstantin/personal/CalibrationProject/calib_output/azure2s10_extrinsics.npy")

calib_intrinsics1 = np.load("/home/konstantin/personal/CalibrationProject/calib_output/azure_intrinsics.npy", allow_pickle=True).item()
calib_intrinsics2 = np.load("/home/konstantin/personal/CalibrationProject/calib_output/s10_intrinsics.npy", allow_pickle=True).item()

FileNotFoundError: [Errno 2] No such file or directory: '/home/konstantin/personal/CalibrationProject/calib_output/azure2s10_extrinsics.npy'

In [104]:
# T = np.array([[ 0.99741621, -0.02765771, -0.06630205,  0.20646171],
#               [ 0.01825322,  0.99019896, -0.13846602, -0.01545667],
#               [ 0.06948188,  0.13689803,  0.98814533, -0.02928021],
#               [ 0.,          0.,          0.,          1.        ]]) # samsung -> azure

# inv_T = np.linalg.inv(T) # azure -> samsung

In [105]:
# azure_intrinsics = np.array([[503.9581604003906, 0.0, 325.4591064453125],
#                        [0.0, 504.0079345703125, 340.33404541015625],
#                        [0.0, 0.0, 1.0]])
# azure_dist_coeff = np.array([5.268435478210449, 3.5621509552001953, -6.14773682627856e-07, -4.438921678229235e-05, 0.1832902431488037, 5.597447872161865, 5.32110071182251, 0.9697896838188171])

In [106]:
from utils.io_utils import get_images, get_pointclouds, get_depth, pointcloudify_depths

samsung_data_path = "/home/konstantin/datasets/bandeja-sequence/2021_02_17/2021-02-17-16-07-15_chessboard_plane/smartphone_video_frames"
azure_data_path = "/home/konstantin/datasets/bandeja-sequence/2021_02_17/2021-02-17-16-07-15_chessboard_plane/_azure_depth_image_raw"

# Explicitly provide matching pairs and mapping
samsung_file_names = ['2012475554907.png', '2012742135368.png']
azure_file_names = ['2012410106602.npy', '2012810117602.npy']
mapping = {'2012410106602.npy': '2012475554907.png',
           '2012810117602.npy': '2012742135368.png'}

samsung_images = get_images(samsung_data_path, file_name_list=samsung_file_names)
azure_depths = get_depth(azure_data_path, file_name_list=azure_file_names)
azure_pointclouds = pointcloudify_depths(azure_depths, azure_intrinsics, azure_dist_coeff)

In [107]:
t_azure_pointclouds = {}

for key, pcd in azure_pointclouds.items():
    pcd_h = np.concatenate((pcd, np.ones((len(pcd), 1))), axis=-1)
    t_pcd = (inv_T @ pcd_h.transpose()).transpose()[:, :3]
    
    t_azure_pointclouds[key] = t_pcd

In [123]:
from scipy.interpolate import griddata
import cv2

vis_t_azure_pointclouds = {}

for key, pcd in t_azure_pointclouds.items():
    image_points = (intrinsics @ pcd.transpose()).transpose()
    image_points = image_points[:, :2] / np.clip(image_points[:, 2], a_min=1e-8, a_max=None).reshape(-1, 1)
    
    image = samsung_images[mapping[key]]
#     image = cv2.undistort(image, intrinsics, dist_coeff, None, undist_intrinsics)
    # TODO. A workaround, use torch
    image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])    
    
    grid_x, grid_y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    grid = np.concatenate([np.expand_dims(grid_x, -1),
                           np.expand_dims(grid_y, -1)], axis=-1)
    
    pcd_color = griddata(grid.reshape(-1, 2), image.reshape(-1), image_points, method='nearest', fill_value=0) / 255.0
    
    open3d_pcd = open3d.geometry.PointCloud()
    open3d_pcd.points = open3d.utility.Vector3dVector(pcd)
    open3d_pcd.colors = open3d.utility.Vector3dVector(np.repeat(pcd_color.reshape(-1, 1), 3, axis=-1))
    
    vis_t_azure_pointclouds[key] = open3d_pcd

In [124]:
vis_t_azure_pointclouds

{'2012410106602.npy': PointCloud with 368640 points.,
 '2012810117602.npy': PointCloud with 368640 points.}

In [126]:
open3d.io.write_point_cloud('../calib_output/test.pcd', vis_t_azure_pointclouds['2012810117602.npy'])

True

In [103]:
# # Load pointcloud from azure and its corresponding image from samsung

# pnp_results = np.load('../calib_output/pnp_results.npy', allow_pickle=True).item()
# pnp_keys = list(pnp_results.keys())

# azure_depth_folder_path = "/home/konstantin/datasets/bandeja-sequence/2021_02_17/2021-02-17-16-07-15_chessboard_plane/_azure_depth_image_raw"
# azure_depth_names = sorted(os.listdir(azure_depth_folder_path), key=str.lower)[:-2]

In [8]:
# pnp_timestamps = np.array([int(i) for i in pnp_keys])
# azure_depth_timestamps = np.array([int(i.split('.')[0]) for i in azure_depth_names])

In [9]:
# diff = np.abs(pnp_timestamps.reshape(-1, 1) - azure_depth_timestamps.reshape(1, -1))
# nn_indices = np.argmin(diff, axis=-1)

In [10]:
# sel_azure_depth_names = [azure_depth_names[i] for i in nn_indices]

In [11]:
# pnp_keys

['2012475554907',
 '2012742135368',
 '2013008706906',
 '2013275281406',
 '2013541857252',
 '2013808503213',
 '2014075005136',
 '2014341579828',
 '2014608154828',
 '2014874727520',
 '2015141302636',
 '2015407876712',
 '2015674451712',
 '2015941023135',
 '2016207600135',
 '2016474175058',
 '2016740751019',
 '2017007322673',
 '2017273897480',
 '2017540470326',
 '2017807045441',
 '2018073621480',
 '2018340200018',
 '2018606771056',
 '2018873345017',
 '2019139939633',
 '2019406492556',
 '2019673066748',
 '2019939644440',
 '2020206217901',
 '2020472794439',
 '2020739444785',
 '2021005996247',
 '2021272516977',
 '2021539091439',
 '2021805667169',
 '2022072241977',
 '2022338814477',
 '2022605388938',
 '2022871968630',
 '2023138538784',
 '2023405112322',
 '2023671686399',
 '2023938255937',
 '2024204841629',
 '2024471408475',
 '2024737988667',
 '2025004556475',
 '2025271137167',
 '2025537711859',
 '2025804367705',
 '2026070857435',
 '2026337434704',
 '2026604004204',
 '2026870588858',
 '20271371

In [12]:
# sel_azure_depth_names

['2012410106602.npy',
 '2012810117602.npy',
 '2013010106602.npy',
 '2013210117602.npy',
 '2013610106602.npy',
 '2013810117602.npy',
 '2014010006602.npy',
 '2014410117602.npy',
 '2014610106602.npy',
 '2014810106602.npy',
 '2015210106602.npy',
 '2015410106602.npy',
 '2015610106602.npy',
 '2016010106602.npy',
 '2016209995602.npy',
 '2016409995602.npy',
 '2016810106602.npy',
 '2017010117602.npy',
 '2017210106602.npy',
 '2017610106602.npy',
 '2017810106602.npy',
 '2018010117602.npy',
 '2018410095602.npy',
 '2018610106602.npy',
 '2018810106602.npy',
 '2019210106602.npy',
 '2019410106602.npy',
 '2019610117602.npy',
 '2020010117602.npy',
 '2020210117602.npy',
 '2020410117602.npy',
 '2020809995602.npy',
 '2021009995602.npy',
 '2021210117602.npy',
 '2021610106602.npy',
 '2021810106602.npy',
 '2022010106602.npy',
 '2022409995602.npy',
 '2022609995602.npy',
 '2022810106602.npy',
 '2023210117602.npy',
 '2023410106602.npy',
 '2023610117602.npy',
 '2024010106602.npy',
 '2024210106602.npy',
 '20244101